In [1]:
import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import os,sys
import tqdm
import pickle
import time
from concurrent.futures import ThreadPoolExecutor  
import concurrent.futures

In [2]:
def get_systemcall_name_perf(line):
    l = re.split(r' |\( |\)', line)
    l = list(filter(lambda a: a != '', l))
    # print(l)
    if len(l) < 6:
        return None
    timestamp = l[0]
    time_cost = l[1]
    pid = l[4]
    if l[5] == '...':
        if timestamp == '0.000':
            return None
        else:
            syscall = l[7].split('(')[0]
    else:
        syscall = l[5].split('(')[0]
    # print(timestamp,time_cost, pid,  syscall)
    return [pid, timestamp, syscall, time_cost]

In [4]:
filename = 'test.txt'

In [5]:
with open(filename, 'r') as f, open(filename+'.csv', 'w') as outp:
    for line in f:
        res = get_systemcall_name_perf(line)
        if res != None:
            [pid, timestamp, syscall, time_cost] = res
            outp.write('{},{},{},{}\n'.format(pid, timestamp, syscall, time_cost))
    f.close()
    outp.close()

In [6]:
rootPath ='D:/mt_data/pi3/'
perfdataPath = rootPath+'new_t1/'
files = os.listdir(perfdataPath)

In [7]:
len(files)

3600

In [16]:
def get_systemcall_name_perf(line):
    l = re.split(r' |\( |\)', line)
    l = list(filter(lambda a: a != '', l))
    # print(l)
    if len(l) < 6:
        return None
    timestamp = l[0]
    time_cost = l[1]
    pid = l[4]
    try:
        if l[5] == '...':
            if timestamp == '0.000':
                return None
            else:
                syscall = l[7].split('(')[0]
        else:
            syscall = l[5].split('(')[0]
    except:
        print(l)
        syscall = None
    # print(timestamp,time_cost, pid,  syscall)
    return [pid, timestamp, syscall, time_cost]

In [17]:
def change_time(df):
    index= 0
    time = 0
    for index,time in enumerate(df['timestamp']):
        if time != 0:
            break
    for j in range(0,index):
        df.loc[j, 'timestamp'] = time
    return df

In [18]:
par = tqdm.tqdm(total=len(files), ncols=100)
for fi in files:
    if 'txt' in fi:
        par.update(1)
        syscall_trace = []
        with open(perfdataPath + fi, 'r') as f:
            for line in f:
                res = get_systemcall_name_perf(line)
                if res != None:
                    syscall_trace.append(res)
                # else: 
                #     print(fi)
            f.close()
        df = pd.DataFrame(syscall_trace)
        df.columns=['pid', 'timestamp', 'syscall', 'time_cost']
        path = rootPath + 'trace_t1/' + fi
        df.to_csv(path, index=None)
        # print(fi, set(df['syscall']))
par.close()

 18%|██████████▋                                               | 660/3600 [15:21<1:04:27,  1.32s/it]

['336366698.794', '2.446', 'ms', ':', 'es_sensor/24564', '...', '[c']


 75%|██████████████████████████████████████████▊              | 2705/3600 [1:02:35<21:03,  1.41s/it]

['458303234.736', '0.115', 'ms', ':', 'es_sensor/30052', '...', '[continu']


100%|█████████████████████████████████████████████████████████| 3600/3600 [1:23:11<00:00,  1.39s/it]


In [19]:
dataPath = rootPath + 'trace_t1/'

In [23]:
def split_data_by_time(savePath, malware_type, pid, rounds,df,time_window):
    start = 0
    end = 0
    ti =  df['timestamp'].tolist()
    start_time = ti[0]
    end_time = 0
    index = 0
    for i in range(0, len(df)):
        end = i
        end_time=ti[end]
        if end_time - start_time >= time_window * 1000:
            fileName = '{}{}_{}_{}_{}_{}_{}.csv'.format(savePath,malware_type,pid, rounds, str(index),str(time_window),str(start_time))
            d = df[start:end]
            d.to_csv(fileName,index=None)
            index += 1
            start = end
            start_time = end_time         

In [20]:
def get_maltype_from_name(filename):
    ns = filename.split('_')
    malware_type = ''
    pid = ''
    rounds = ''
    if ns[1] != 'v2':
        malware_type =  ns[0]
        pid=  ns[1]
        rounds =  ns[2]
    else:
        malware_type = ns[0]+ns[1]
        id=  ns[2]
        rounds =  ns[3]
    return malware_type, pid, rounds

In [25]:
# tws = [10, 15, 16, 17, 20, 25, 30, 35, 40, 50, 60]
tws = [60]
datafiles = os.listdir(dataPath)
newdataPaht= rootPath+'pi4_4G_splited1/'
for tw in tws:
    ts = newdataPaht+ str(tw) +'/'  
    ls = os.listdir(newdataPaht)
    if str(tw) not in ls:
        os.mkdir(ts)
        ts = newdataPaht + str(tw) +'/'  
    par = tqdm.tqdm(total = len(datafiles), ncols=100)
    for fi in datafiles:
        if '.txt' in fi:
            par.update(1)
            malware_type, pid, rounds = get_maltype_from_name(fi)
            df = pd.read_csv(dataPath+fi)
            time_window = tw
            split_data_by_time(ts, malware_type, pid, rounds,df,time_window)
    par.close()


100%|█████████████████████████████████████████████████████████████| 600/600 [29:14<00:00,  2.92s/it]


In [49]:
def get_syscall_dict(ngrams_dict):
    syscall_dict = {}
    i = 0
    for ngram in ngrams_dict:
        if len(ngram.split()) == 1:
            syscall_dict[ngram] = i
            i+=1
    return syscall_dict



In [50]:
def create_vectorizers(corpus, ngram):
    syscall_dict = {}
    ngrams_dict = {}
    # countvectorizer = CountVectorizer().fit(corpus)
    # syscall_dict = countvectorizer.vocabulary_
    countvectorizer = CountVectorizer(ngram_range=(1, ngram)).fit(corpus)
    print('create count vectorizer finished')
    ngrams_dict = countvectorizer.vocabulary_
    syscall_dict = get_syscall_dict(ngrams_dict)
    tfidfvectorizer = TfidfVectorizer(ngram_range=(1, ngram), vocabulary=ngrams_dict).fit(corpus)
    print('create tf-idf vectorizer finished')
    hashingvectorizer = HashingVectorizer(n_features=2**5).fit(corpus)  
    print('create hashing vectorizer finished')
    return syscall_dict, ngrams_dict, countvectorizer, tfidfvectorizer, hashingvectorizer

In [51]:
tw = 50
# rawdataPath = rootPath+'data_1/'+str(tw)+'/'
rawdataPath ='F:/temp/50/'
rawFileNames = os.listdir(rawdataPath)
len(rawFileNames)

3321

In [52]:
def from_trace_to_longstr(syscall_trace):
        tracestr = ''
        for syscall in syscall_trace:
            tracestr += syscall + ' '
        # print(tracestr)
        return tracestr

In [53]:
rawdatas = dict()
for n in range(0,len(rawFileNames),1000):
    start = n
    if start + 1000 < len(rawFileNames):
        end = start + 1000
    else:
        end = len(rawFileNames)
    rawdatas[n] = rawFileNames[start:end]

In [54]:
corpus_dataframes, corpuses = {}, {}

In [55]:
def read_all_rawdata(corpus_dataframe,corpus, rawdataPath, rawFileNames):    
    pool = ThreadPoolExecutor  (max_workers = 16)
    def read_file(inputFilePath):
        trace = pd.read_csv(inputFilePath)
        tr = trace['syscall'].tolist()             
        longstr = from_trace_to_longstr(tr)
        return (trace,longstr)
        # print(inputFilePath)
    def asyn_page(filenames):
        future_to_url  = dict()
        for i, url in enumerate(filenames):
            t = pool.submit(read_file, url)
            future_to_url[t] = url               
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                (trace,longstr) = data
                corpus_dataframe.append(trace)
                corpus.append(longstr)
            except Exception as exc:
                print('%r generated an exception: %s' % (filenames, exc))
    
    par = tqdm.tqdm(total = len(rawFileNames), ncols=100)
    # i = 0
    start, end = 0,0
    for n in range(0,len(rawFileNames),16):
        par.update(16)
        start = n
        if start + 16 < len(rawFileNames):
            end = start + 16
        else:
            end = len(rawFileNames)
        filenames = [rawdataPath + rawFileNames[i] for i in range(start, end)]
        asyn_page(filenames)
    par.close()
    pool.shutdown()
    print("Sub-process(es) done.")
    return corpus_dataframe, corpus

In [56]:
for rfs in rawdatas:
    corpus_dataframe, corpus = [],[]
    corpus_dataframe, corpus = read_all_rawdata(corpus_dataframe,corpus, rawdataPath, rawdatas[rfs])
    corpus_dataframes[rfs] = corpus_dataframe
    corpuses[rfs] = corpus


  3%|█▉                                                           | 32/1000 [00:01<00:52, 18.40it/s]C:\Python39\lib\concurrent\futures\thread.py:52: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  result = self.fn(*self.args, **self.kwargs)
1008it [03:00,  5.59it/s]


Sub-process(es) done.


1008it [03:39,  4.59it/s]


Sub-process(es) done.


1008it [04:02,  4.15it/s]


Sub-process(es) done.


336it [01:07,  4.95it/s]

Sub-process(es) done.


In [57]:
corpus_dataframe, corpus = [],[]
for rfs in corpus_dataframes:
    corpus_dataframe += corpus_dataframes[rfs]
    corpus += corpuses[rfs]

In [58]:
syscall_dict, ngrams_dict, countvectorizer, tfidfvectorizer, hashingvectorizer = create_vectorizers(corpus, 3)

create count vectorizer finished
create tf-idf vectorizer finished
create hashing vectorizer finished


In [59]:
len(ngrams_dict)

505

In [60]:
len(syscall_dict)

16

In [61]:
def create_onehot_encoding(total, index):
    onehot = []
    for i in range(0, total):
        if i == index:
            onehot.append(1)
        else:
            onehot.append(0)
    return onehot

In [62]:
def add_unk_to_dict(syscall_dict):
    total = len(syscall_dict)
    syscall_dict['unk'] = total
    syscall_dict_onehot = dict()
    for sc in syscall_dict:
        syscall_dict_onehot[sc] = create_onehot_encoding(total+1, syscall_dict[sc])
    return syscall_dict, syscall_dict_onehot

In [63]:
syscall_dict, syscall_dict_onehot = add_unk_to_dict(syscall_dict)

In [64]:
def replace_with_unk(syscall_trace, syscall_dict):
    for i, sc in enumerate(syscall_trace):
        if sc.lower() not in syscall_dict:
            syscall_trace[i] = 'unk'
    return syscall_trace

In [65]:
frequency_features = countvectorizer.transform(corpus)
tfidf_features = tfidfvectorizer.transform(corpus)
hashing_features = hashingvectorizer.transform(corpus)

In [66]:
def trace_onehot_encoding(trace, syscall_dict_onehot):
    encoded_trace = []
    for syscall in trace:
        syscall = syscall.lower()
        if syscall.lower() in syscall_dict_onehot:
            one_hot = syscall_dict_onehot[syscall]
        else:
            syscall = 'UNK'
            one_hot = syscall_dict_onehot[syscall]
        encoded_trace.append(one_hot)
    return encoded_trace

In [67]:
def find_all_head(trace, head):
    starts, ends,se = [], [], []

    for i,s in enumerate(trace):
        if s == head:
            start=i
            starts.append(start)
            if len(starts) > 1:
                end = starts[-1] 
                ends.append(end)
        if i == len(trace)-1:
            end = len(trace)
            ends.append(end)
    se = [(starts[i], ends[i]) for i in range(0, len(starts))]
    return se

In [68]:
def get_distance( trace,head,tails):
    se = find_all_head(trace, head)
    mw = 12
    pool = ThreadPoolExecutor  (max_workers = mw)
    distances = []
    res = dict()
    def return_distance(tails, sort):
        distance = dict()
        for tail in tails:
            d = 0
            for j,t in enumerate(sort):
                if t==tail:
                    d += 1/(j)
            distance[tail] = d
        return distance

    def asyn_page(tails, sorts):
        future_to_url  = dict()
        for i, url in enumerate(sorts):
            t = pool.submit(return_distance, tails=tails, sort=url)
            future_to_url[t] = url               
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                distance = data
                distances.append(distance)
            except Exception as exc:
                print('generated an exception: %s' % (exc))
    start, end = 0,0
    for n in range(0, len(se), mw):
        start = n
        if start + mw < len(rawFileNames):
            end = start + mw
        else:
            end = len(rawFileNames)            
        sorts = [trace[s:e] for (s, e) in se[start:end]]
        asyn_page(tails, sorts)
    pool.shutdown()
    # print("Sub-process(es) done.")
    ds = pd.DataFrame(distances)
    for tail in tails:
        if tail in ds:
            res[(head, tail)] = sum(ds[tail])
        else:
            res[(head, tail)] = 0
    return res

In [102]:
# def get_dependency_graph(trace,term_dict):
#     dp = []
#     for head in term_dict:
#         dp_ = []
#         for tail in term_dict:
#             if head == tail:
#                 dp_.append(0)
#             else:
#                 distance = get_distance(trace,head,tail)
#                 dp_.append(distance)
#         dp.append(dp_)
#     return dp

In [103]:
# def get_dependency_graph(trace,term_dict):
#     dp_keys = []
#     dp = dict()
#     for head in term_dict:
#         for tail in term_dict:
#             dp_key = (head,tail)
#             dp_keys.append(dp_key)
#             dp[dp_key] = 0
#     mw = 12
#     pool = ThreadPoolExecutor  (max_workers = mw)
      
#     def asyn_page(trace, cf):
#         future_to_url  = dict()
#         for i, url in enumerate(cf):
#             (head,tail) = url
#             if head == tail:
#                 continue
#             else:
#                 t = pool.submit(get_distance, trace,head,tail)
#             future_to_url[t] = url               
#         for future in concurrent.futures.as_completed(future_to_url):
#             url = future_to_url[future]
#             try:
#                 data = future.result()
#                 distance = data
#                 dp[url] = distance
#             except Exception as exc:
#                 print('generated an exception: %s' % ( exc))    
#     # i = 0
#     start, end = 0,0
#     for n in range(0,len(dp_keys),mw):
#         start = n
#         if start + mw < len(dp_keys):
#             end = start + mw
#         else:
#             end = len(dp_keys)
#         cf = dp_keys[start: end]
#         asyn_page(trace, cf)
#     return dp

In [69]:
def get_dependency_graph(trace,term_dict):
    dp = []
    dp_ = {}
    for head in term_dict:
        tails = list(term_dict.keys())
        tails.remove(head)  
        p = get_distance(trace,head,tails)
        dp_ = {**dp_, **p}
    
    for head in term_dict:
        d_ =  []
        for tail in term_dict:
            if head == tail:
                d_.append(0)
            else:
                d_.append(dp_[(head, tail)])
        dp.append(d_)
    return dp

In [70]:
def get_dict_sequence(trace,term_dict):
    dict_sequence = []
    for syscall in trace:
        if syscall in term_dict:
            dict_sequence.append(term_dict[syscall])
        else:
            dict_sequence.append(term_dict['unk'])
    return dict_sequence


In [106]:
# one_hot_features = []
# dependency_graph_features = []
# dict_sequence_features = []

# par = tqdm.tqdm(total = len(corpus_dataframe), ncols=100)
# for trace in corpus_dataframe:
#     par.update(1)
#     one_hot = []
#     dependency_graph = []
#     dict_sequence = []
#     syscall_trace = replace_with_unk(trace['syscall'].to_list(), syscall_dict)
#     syscall_one_hot =  trace_onehot_encoding(syscall_trace, syscall_dict_onehot)
#     dependency_graph = get_dependency_graph(syscall_trace,syscall_dict)
#     dict_sequence=get_dict_sequence(syscall_trace,syscall_dict)

#     one_hot_features.append(syscall_one_hot)
#     dependency_graph_features.append(dependency_graph)  
#     dict_sequence_features.append(dict_sequence) 
# par.close()

In [71]:
def get_sequence_features(corpus_dataframe, syscall_dict, syscall_dict_onehot):
    one_hot_features = []
    dependency_graph_features = []
    dict_sequence_features = []
    pool = ThreadPoolExecutor  (max_workers = 16)

    def get_features(trace, syscall_dict, syscall_dict_onehot):
        syscall_one_hot = []
        dependency_graph = []
        dict_sequence = []
        syscall_trace = replace_with_unk(trace['syscall'].to_list(), syscall_dict)
        syscall_one_hot =  trace_onehot_encoding(syscall_trace, syscall_dict_onehot)
        dependency_graph = get_dependency_graph(syscall_trace,syscall_dict)
        dict_sequence=get_dict_sequence(syscall_trace,syscall_dict)
        return (syscall_one_hot, dependency_graph, dict_sequence)

    def asyn_page(cf):
        future_to_url  = dict()
        for i, url in enumerate(cf):
            t = pool.submit(get_features, url, syscall_dict, syscall_dict_onehot)
            future_to_url[t] = url               
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                (syscall_one_hot, dependency_graph, dict_sequence) = data
                one_hot_features.append(syscall_one_hot)
                dependency_graph_features.append(dependency_graph)
                dict_sequence_features.append(dict_sequence)
            except Exception as exc:
                print('generated an exception: %s' % ( exc))


    par = tqdm.tqdm(total = len(corpus_dataframe), ncols=100)
    # i = 0
    start, end = 0,0
    for n in range(0,len(corpus_dataframe),16):
        par.update(16)
        start = n
        if start + 16 < len(corpus_dataframe):
            end = start + 16
        else:
            end = len(corpus_dataframe)
        cf = corpus_dataframe[start: end]
        asyn_page(cf)
    par.close()   
    return one_hot_features, dependency_graph_features, dict_sequence_features

In [72]:
one_hot_features, dependency_graph_features, dict_sequence_features = get_sequence_features(corpus_dataframe, syscall_dict, syscall_dict_onehot)

3328it [1:21:48,  1.48s/it]


In [73]:
maltype = []
ids = []
for fi in rawFileNames:
    fis = fi.split('_')
    fn = fis[0]
    i = '{}_{}_{}'.format(fis[0], fis[2], fis[3])
    maltype.append(fn)
    ids.append(i)

In [110]:
temppath  ='F:/temp/{}'.format(tw)
loc=open(temppath+'ids.pk','wb')
pickle.dump(ids,loc)
loc=open(temppath+'maltype.pk','wb')
pickle.dump(maltype,loc)
loc=open(temppath+'frequency_features.pk','wb')
pickle.dump(frequency_features,loc)
loc=open(temppath+'tfidf_features.pk','wb')
pickle.dump(tfidf_features,loc)
loc=open(temppath+'hashing_features.pk','wb')
pickle.dump(hashing_features,loc)
loc=open(temppath+'hashing_features.pk','wb')
pickle.dump(hashing_features,loc)

loc=open(temppath+'corpus_dataframe.pk','wb')
pickle.dump(corpus_dataframe,loc)

loc=open(temppath+'corpus.pk','wb')
pickle.dump(corpus,loc)
loc=open(temppath+'dependency_graph_features.pk','wb')
pickle.dump(dependency_graph_features,loc)
loc=open(temppath+'one_hot_features.pk','wb')
pickle.dump(one_hot_features,loc)
loc=open(temppath+'dict_sequence_features.pk','wb')
pickle.dump(dict_sequence_features,loc)
loc.close()

In [74]:
dictPath = rootPath +'dicts/pi4_2G/'+str(tw)+'/'
loc=open(dictPath+'countvectorizer.pk','wb')
pickle.dump(countvectorizer,loc)
loc=open(dictPath+'tfidfvectorizer.pk','wb')
pickle.dump(tfidfvectorizer,loc)
loc=open(dictPath+'hashingvectorizer.pk','wb')
pickle.dump(hashingvectorizer,loc)
loc=open(dictPath+'syscall_dict.pk','wb')
pickle.dump(syscall_dict,loc)
loc=open(dictPath+'syscall_dict_onehot.pk','wb')
pickle.dump(syscall_dict_onehot,loc)
loc=open(dictPath+'ngrams_dict.pk','wb')
pickle.dump(ngrams_dict,loc)
loc.close()

In [29]:
# temppath  ='F:/temp/'
dictPath = rootPath +'dicts/'+str(tw)+'/'

# loc=open(temppath+'ids.pk','rb')
# ids = pickle.load(loc)
# loc=open(temppath+'maltype.pk','rb')
# maltype = pickle.load(loc)
# loc=open(temppath+'frequency_features.pk','rb')
# frequency_features = pickle.load(loc)
# loc=open(temppath+'tfidf_features.pk','rb')
# tfidf_features = pickle.load(loc)
# loc=open(temppath+'hashing_features.pk','rb')
# hashing_features = pickle.load(loc)

# loc=open(temppath+'corpus_dataframe.pk','rb')
# corpus_dataframe = pickle.load(loc)
# loc=open(temppath+'corpus.pk','rb')
# corpus = pickle.load(loc)

loc=open(dictPath+'syscall_dict.pk','rb')
syscall_dict = pickle.load(loc)

loc=open(dictPath+'syscall_dict_onehot.pk','rb')
syscall_dict_onehot = pickle.load(loc)

loc=open(dictPath+'ngrams_dict.pk','rb')
ngrams_dict = pickle.load(loc)


loc=open(dictPath+'countvectorizer.pk','rb')
countvectorizer = pickle.load(loc)

loc=open(dictPath+'hashingvectorizer.pk','rb')
hashingvectorizer = pickle.load(loc)

loc=open(dictPath+'tfidfvectorizer.pk','rb')
tfidfvectorizer = pickle.load(loc)


# loc=open(temppath+'dependency_graph_features.pk','rb')
# dependency_graph_features = pickle.load(loc)
# loc=open(temppath+'one_hot_features.pk','rb')
# one_hot_features = pickle.load(loc)
# loc=open(temppath+'dict_sequence_features.pk','rb')
# dict_sequence_features = pickle.load(loc)

In [75]:
encoded_trace_df = pd.DataFrame([ids, maltype,frequency_features.toarray() ,tfidf_features.toarray(),hashing_features.toarray(), dependency_graph_features, one_hot_features, dict_sequence_features] ).transpose()
encoded_trace_df.columns = ['ids', 'maltype',  'system calls frequency' ,'system calls tfidf','system calls hashing', 'system calls dependency graph', 'one hot encoding', 'dict index encoding']

In [76]:
temppath  ='F:/temp/{}/'.format(tw)
encoded_trace_df.to_pickle(temppath+'encoded_bow.pkl')

In [77]:
tw = 50
resultsPath = 'D:/git/IoT_Sensors_Security_Analysis/results/pi4_2G/tw_{}_turn_1/'.format(tw)

In [116]:
# temppath  ='F:/temp/'
# encoded_trace_df = pd.read_pickle(temppath+'encoded_bow.pkl')

In [117]:
# tfidfvectorizer1 = TfidfVectorizer(ngram_range=(1, 5),vocabulary=ngrams_dict)
# with open('tfidfvectorizer.pk', 'wb') as fin:
#     pickle.dump(tfidfvectorizer1, fin)

In [78]:
features = ['system calls frequency' ,'system calls tfidf','system calls hashing', 'system calls dependency graph', 'one hot encoding', 'dict index encoding' ]
malwares=["delay", "disorder", "freeze", "hop", "mimic", "noise", "repeat", "spoof"]
# features = ['hashing_features' ,'dependency_graph_features', 'one_hot_features']
# malwares=["delay"]

In [79]:
normal = encoded_trace_df[encoded_trace_df.maltype=='normal']

In [80]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import Nystroem
from sklearn import metrics
from sklearn.svm import OneClassSVM
import time

In [81]:
def reshape_matrix(matrix_list):
    new_list = [np.array(i).reshape(-1) for i in matrix_list]
    return new_list

In [82]:
def padding_onehot(onehot_list, padding):
    new_list = []
    for onehot in onehot_list:
        if len(onehot) > padding:
            onehot = np.array(onehot[0:padding])
            new_list.append(onehot)
        else:
            onehot =np.pad(onehot, [(0, padding-len(onehot)), (0, 0)], mode='constant', constant_values=0)
            new_list.append(onehot)
    new_list = reshape_matrix(new_list)
    return new_list
    

In [83]:
def padding_dictencoding(dictencoding_list, padding):
    new_list = []
    for onehot in dictencoding_list:
        if len(onehot) > padding:
            onehot = np.array(onehot[0:padding])
            new_list.append(onehot)
        else:
            onehot =np.pad(onehot, [(0, padding-len(onehot))], mode='constant', constant_values=0)
            new_list.append(onehot)
    return new_list

In [84]:
def train_models(feature, normal):
    X = normal[feature].tolist()
    y = normal['maltype'].tolist()
    # mlb = LabelBinarizer()
    
    y = np.zeros(len(X))
    # h = .02  # step size in the mesh
    outliers_fraction = 0.15
    nu = 0.05
    clfs = []
    results = []
    preds = []
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3, random_state=42)
    y_val_2 = [1 if i==0 else -1 for i in y_val]

    X_tv = []
    pca = PCA(n_components=100)
    if feature =='system calls dependency graph':
        X_train = reshape_matrix(X_train)
        X_val = reshape_matrix(X_val)    
        X_tv=[(X_train,X_val)]
        features = [feature]
    elif feature =='one hot encoding':
        X_train_55000 = padding_onehot(X_train, 55000)
        X_val_55000 = padding_onehot(X_val, 55000)
        X_train_pca = pca.fit_transform(X_train_55000)
        X_val_pca = pca.transform(X_val_55000)
        X_tv = [(X_train_pca,X_val_pca)]
        features = [feature]
    elif feature =='dict index encoding':
        X_train_55000 = padding_dictencoding(X_train, 55000)
        X_val_55000 = padding_dictencoding(X_val, 55000)
        X_train_pca = pca.fit_transform(X_train_55000)
        X_val_pca = pca.transform(X_val_55000)
        X_train_10000 = padding_dictencoding(X_train, 10000)
        X_val_10000 = padding_dictencoding(X_val, 10000)
        X_tv = [(X_train_pca,X_val_pca),(X_train_10000,X_val_10000)]
        features = [feature+'-pca', feature]
    elif feature =='system calls hashing':
        X_tv=[(X_train,X_val)]
        features = [feature]
    else:
        X_train_pca = pca.fit_transform(X_train)
        X_val_pca = pca.transform(X_val)
        X_tv = [(X_train_pca,X_val_pca),(X_train,X_val)]
        features = [feature+'-pca', feature]

    for i,(X_train,X_val)  in enumerate(X_tv):
        feature = features[i]
        result = []
        pred = dict()
        classifiers = {
                "Robust covariance": EllipticEnvelope(contamination=outliers_fraction),
                "One-Class SVM": OneClassSVM(nu=outliers_fraction, kernel="rbf",gamma=0.1),
                "SGD One-Class SVM": SGDOneClassSVM(nu=nu, shuffle=True, fit_intercept=True, random_state=42, tol=1e-4),
                "Isolation Forest": IsolationForest(contamination=outliers_fraction,random_state=42),
            }
        for name in classifiers:
            clf = classifiers[name]
            t1 =time.time()
            res = dict()
            clf.fit(X_train)
            t2 =time.time()
            y_pred = clf.predict(X_val)
            score = metrics.accuracy_score(y_val_2,y_pred)

            pred['valid_'+  feature + '_' + name] = y_pred
            t = t2 -t1
            res['Model'] ='valid_' + feature + '_' + name
            res['Accuracy'] = score
            res['Training time'] = t
            result.append(res)
            print('Model: {}, accuracy score: {}, training time is: {} seconds'.format(res['Model'], score, t))
        results.append(result)
        preds.append(pred)
        clfs.append(classifiers)
    pca = pca
    return clfs, results, preds, pca

In [85]:
def test_models(malware, feature, clfs, pca):
    dfs = encoded_trace_df[encoded_trace_df.maltype==malware]
    X_test = dfs[feature].tolist()
    X_tv = [] 
    if feature =='system calls dependency graph':
        X_test = reshape_matrix(X_test)    
        X_tv=[X_test]
        features = [feature]
    elif feature =='one hot encoding':
        X_test_55000 = padding_onehot(X_test, 55000)
        X_test_pca = pca.transform(X_test_55000)
        X_tv = [X_test_pca]
        features = [feature]
    elif feature =='dict index encoding':
        X_test_55000 = padding_dictencoding(X_test, 55000)       
        X_test_pca = pca.transform(X_test_55000)
        X_test_10000 = padding_dictencoding(X_test, 10000)       
        X_tv = [X_test_pca, X_test_10000]
        features = [feature+'-pca', feature]
    elif feature =='system calls hashing':
        X_tv=[X_test]
        features = [feature]
    else:

        X_test_pca = pca.transform(X_test)
        X_tv = [X_test_pca, X_test]
        features = [feature+'-pca', feature]

    results = []
    preds = []
    y_test = np.ones(len(X_test))
    y_test_2 = [1 if i==0 else -1 for i in y_test]

    for i,classifiers in enumerate(clfs):
        feature = features[i]
        X_test = X_tv[i]
        result = []
        pred = dict()    
        for name in classifiers:
            res = dict()
            clf = classifiers[name]
            t1 =time.time()
            if 'anomaly detection' in name:
                y_pred = clf.predict(X_test)
                t2 =time.time()
                score = metrics.accuracy_score(y_test_2,y_pred)
            else:
                y_pred = clf.predict(X_test)
                t2 =time.time()
                score = metrics.accuracy_score(y_test,y_pred)
            t = t2 -t1
            pred[malware +'_' + feature + '_' + name] = y_pred
            res['Model'] = malware +'_' + feature + '_' + name
            res['Accuracy'] = score
            res['Testing time'] = t
            result.append(res)
            print('Model: {}, accuracy score: {}, testing time is: {} seconds'.format( res['Model'], score, t))
        results.append(result)
        preds.append(pred)
    return results, preds       

In [86]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [87]:
resultsdict = dict()
predsdict = dict()
classifiersdict = dict()
pcas = dict()
for feature in features:
    #train stage
    clfs, results, preds, pca = train_models(feature, normal)
    
    resultsdict[feature+'_validation'] = results
    predsdict[feature+'_validation'] = preds
    classifiersdict[feature] = clfs
    pcas[feature] = pca
    # testing stage
    for malware in malwares:
        results, preds = test_models(malware, feature, clfs, pca)
        resultsdict[malware +'_' + feature] = results
        predsdict[malware +'_' + feature] = preds
        
loc=open(resultsPath+'classifiers.pk','wb')
pickle.dump(classifiersdict,loc)
loc=open(resultsPath+'results.pk','wb')
pickle.dump(resultsdict,loc)
loc=open(resultsPath+'preds.pk','wb')
pickle.dump(predsdict,loc)   
loc=open(resultsPath+'pcas.pk','wb')
pickle.dump(pca,loc)   

Model: valid_system calls frequency-pca_Robust covariance, accuracy score: 0.828125, training time is: 3.89487361907959 seconds
Model: valid_system calls frequency-pca_One-Class SVM, accuracy score: 0.0, training time is: 0.010002374649047852 seconds
Model: valid_system calls frequency-pca_SGD One-Class SVM, accuracy score: 0.359375, training time is: 0.002999544143676758 seconds
Model: valid_system calls frequency-pca_Isolation Forest, accuracy score: 0.78125, training time is: 0.13303065299987793 seconds
Model: valid_system calls frequency_Robust covariance, accuracy score: 0.8671875, training time is: 19.110288619995117 seconds
Model: valid_system calls frequency_One-Class SVM, accuracy score: 0.0, training time is: 0.020003795623779297 seconds
Model: valid_system calls frequency_SGD One-Class SVM, accuracy score: 1.0, training time is: 0.002000570297241211 seconds
Model: valid_system calls frequency_Isolation Forest, accuracy score: 0.8125, training time is: 0.2270512580871582 seco

In [88]:
rd = []
for rs in resultsdict:
    for r in resultsdict[rs]:
        for s in r:
            rd.append(s)

In [89]:
rd = pd.DataFrame(rd)

In [90]:
md = [i.split('_') for i in rd['Model']]

In [91]:
md = pd.DataFrame(md)
md.columns  = ['Dataset','Features','Model']

In [92]:
md['Model']

0      Robust covariance
1          One-Class SVM
2      SGD One-Class SVM
3       Isolation Forest
4      Robust covariance
             ...        
319     Isolation Forest
320    Robust covariance
321        One-Class SVM
322    SGD One-Class SVM
323     Isolation Forest
Name: Model, Length: 324, dtype: object

In [93]:
nrd=pd.DataFrame([md['Dataset'],md['Features'],md['Model'], rd['Accuracy']]).transpose()

In [94]:
nrd

,Dataset,Features,Model,Accuracy
0,valid,system calls frequency-pca,Robust covariance,0.828125
1,valid,system calls frequency-pca,One-Class SVM,0.0
2,valid,system calls frequency-pca,SGD One-Class SVM,0.359375
3,valid,system calls frequency-pca,Isolation Forest,0.78125
4,valid,system calls frequency,Robust covariance,0.867188
...,...,...,...,...
319,spoof,dict index encoding-pca,Isolation Forest,0.966321
320,spoof,dict index encoding,Robust covariance,0.65285
321,spoof,dict index encoding,One-Class SVM,0.093264
322,spoof,dict index encoding,SGD One-Class SVM,1.0


In [95]:
nrd.to_csv(resultsPath+'results.csv',index=None)